In [12]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
from umap import UMAP
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import torch.nn as nn
import torch

In [18]:
decoder_out = np.load('../data/heart_endothelial_decoderOutput.npy')
sample = np.load('../data/heart_endothelial_decoder_output.npy')

In [42]:
sample[0][:200]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 16.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0., 73.,  0.,  0., 42.,  0.,  0., 16.,  0.,  0.,  0.,
       27.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  5., 95.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 97.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  5.,  0.,  0., 35., 73.,  0.,  0.,  0.,  0., 57.,  5.,  0.,
        0.,  0., 77.,  5.,  0.,  0.,  0.,  0., 27.,  0.,  0.,  0.,  5.,
        0.,  5.,  0.,  0.,  0.,  0., 35., 35.,  0.,  5.,  0.,  0.,  5.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [43]:
sample[1][:200]

array([ 0., 54.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 87.,  0., 20.,  0.,
        0., 65., 66.,  0.,  0.,  0.,  0.,  0.,  0., 63.,  0.,  0.,  0.,
        0.,  0., 67.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 74.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 58.,  0., 65.,  0.,
        0.,  0.,  0.,  0., 63.,  0., 84., 51., 66.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0., 67., 63.,  0.,  0., 60.,  0.,
       78.,  0., 77., 70.,  0.,  0., 51.,  0.,  0.,  0.,  0.,  0., 43.,
       56., 95.,  7., 45.,  0.,  0.,  0.,  0.,  0.,  0., 45.,  0., 58.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 55.,  0.,  0.,
        0., 84., 47., 76.,  0.,  0., 45., 65., 67.,  0.,  0., 57.,  0.,
       57., 61., 53.,  0.,  0.,  0.,  0., 47.,  0.,  0.,  0., 83., 58.,
        0., 34.,  0., 75., 49., 58.,  0., 58., 49.,  0.,  0., 59., 73.,
        0., 76., 60.,  0.,  0.,  0.,  0., 52.,  0., 80.,  0., 60.,  0.,
       75.,  0., 55., 50., 61.,  0.,  0.,  0.,  0.,  0., 58., 58

In [13]:
s = nn.Softmax(dim=1)
softmax_output = s(torch.tensor(decoder_out))

In [34]:
print(softmax_output[1][53])
print(softmax_output[1][54])

tensor(0.0043)
tensor(0.0342)


In [38]:
softmax_output[0]

tensor([6.8389e-01, 5.2137e-04, 2.1630e-03, 4.9385e-03, 9.4122e-04, 1.7931e-03,
        1.2628e-03, 2.4128e-03, 1.1128e-03, 1.1469e-03, 1.3827e-03, 2.4144e-03,
        1.9200e-03, 6.1298e-04, 7.4922e-04, 2.8117e-04, 1.0531e-03, 5.6845e-04,
        4.3583e-04, 1.4802e-04, 3.5086e-03, 2.7101e-04, 2.6034e-04, 4.7865e-04,
        3.6999e-04, 8.4489e-04, 7.0153e-04, 1.1048e-03, 6.8504e-04, 2.0688e-04,
        3.2956e-04, 5.7597e-04, 1.9970e-04, 1.5615e-03, 1.6487e-03, 5.6055e-04,
        1.3625e-03, 5.5165e-04, 3.7087e-03, 4.2571e-03, 6.3670e-04, 9.8983e-04,
        1.3871e-03, 4.5094e-03, 2.6068e-03, 7.6483e-03, 4.3205e-03, 6.8543e-03,
        7.3478e-03, 1.8031e-03, 3.3371e-03, 1.1764e-02, 4.4240e-03, 3.6802e-03,
        1.3570e-02, 1.2238e-02, 4.2634e-03, 4.6537e-03, 9.3545e-03, 4.9004e-03,
        1.3743e-02, 5.9993e-03, 5.1986e-03, 8.0568e-03, 4.7439e-03, 9.4446e-03,
        3.6064e-03, 1.2583e-02, 3.2218e-03, 6.4941e-03, 3.9360e-03, 3.7266e-03,
        4.4897e-03, 6.2302e-03, 3.6803e-